In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, clear_output
import dataloader
import features
import matplotlib.gridspec
import utils
import librosa 

plt.rcParams['text.usetex'] = True
PATH = "/home/max/ET-TI/Masterarbeit/mirdata/"  # path to datasets
SAMPLERATE = 22050
data = {} # global dictionary that is needed to access data from callback

def update_dropdown_id_options(*args):
    selected_split = dropdown_split.value
    data["dataset"] = dataloader.MIRDataset("beatles",basepath=PATH, use_deep_chroma=True, align_chroma=False, split_nr=selected_split)
    dropdown_id.options = list(data["dataset"].getTrackList().keys())
    dropdown_id.value = list(data["dataset"].getTrackList().keys())[0]

def update_selected_track_id(*args):
    textbox_track_id.value = data["dataset"].getTrackList()[dropdown_id.value]

def on_control_change(change):  
    # change some controls according to updates
    plotData(range_slider.value,beat_alignment.value,chroma_type.value)

def plotData(time_range,align_chroma,chroma_type):
    # Clear the previous output before updating the plot
    with output:
        clear_output(wait=True)
        y,_ = librosa.load(data['path'],offset=time_range[0],duration=(time_range[1]-time_range[0]))

        # define gridspec and axes
        fig = plt.figure(figsize=(14, 7))
        gs = matplotlib.gridspec.GridSpec(4, 3,width_ratios=(40,1,5),height_ratios=(1,1,10,10), hspace=0.1,wspace=0.05)
        ax01 = fig.add_subplot(gs[0,0])
        ax11 = fig.add_subplot(gs[1,0])
        ax21 = fig.add_subplot(gs[2,0])
        ax22 = fig.add_subplot(gs[2,1])
        ax31 = fig.add_subplot(gs[3,0])
        ax32 = fig.add_subplot(gs[3,1:3])

        utils.plotChordAnnotations(ax01,data['target'],time_interval=time_range)
        utils.plotAudioWaveform(ax11,y,time_interval=time_range)
        ax11.set_axis_off()
        # chromagram
        if chroma_type == "librosa":
            chroma = features.librosaChroma(y,hop_length=4096)
        elif chroma_type == "madmom":
            chroma = features.deepChroma(y, data['data_split'])
        elif chroma_type == "CRP":
            N = 2**14
            hop_length = N // 4
            chroma = features.crpChroma(y,n=N,overlap=N-hop_length)
        time_vector = np.linspace(time_range[0], time_range[1], chroma.shape[0], endpoint=False)
        img = utils.plotChromagram(ax21, time_vector, chroma)
        ax21.set_xlabel("")
        ax21.set_xticks([])
        fig.colorbar(img,cax=ax22,cmap="Reds")
        
        # features
        colors = ["blue", "green", "red", "orange", "purple", "grey", "black"]
        ax31.plot(time_vector, features.sumChromaDifferences(chroma),color=colors[0])
        ax31.plot(time_vector, features.angularDeviation(chroma),color=colors[1])
        ax31.plot(time_vector, features.flatness(chroma),color=colors[2])
        ax31.plot(time_vector, features.shannonEntropy(chroma),color=colors[3])
        ax31.plot(time_vector, features.negativeSlope(chroma),color=colors[4])
        ax31.plot(time_vector, features.nonSparseness(chroma),color=colors[5])
        ax31.set_xlim(time_vector[0],time_vector[-1])
        ax31.set_ylim(0,1)
        ax31.grid(True)
        #create legend
        labels = ['Sum of diff.',"Fifth Width","Flatness","Entropy","Linear Slope","Non-Sparseness"]
        legend_handles = [patches.Patch(color=color) for color in colors]
        legend = ax32.legend(legend_handles, labels, loc='center left', bbox_to_anchor=(0, 0.5), title='complexity features', handlelength=1, handletextpad=0.5, fontsize=8, title_fontsize=10, facecolor='lightgray', framealpha=0.8)
        ax32.add_artist(legend)
        ax32.set_axis_off()
        plt.show()

def process(_):
    # load some data here
    with output:
        clear_output(wait=True)
    audiopath,target = data["dataset"][dropdown_id.value]
    data['path'] = audiopath # path to audiofile
    data['target'] = target # ground truth estimation
    data['data_split'] = dropdown_split.value # split for neural networks
    # update and show the controls
    range_slider.max = 120
    controls_container.children = [selection, controls]
    #controls_container.children = [selection, controls, cps_controls]
    plotData((0,10),False,'librosa')

#Create widgets
output = widgets.Output()
dropdown_dataset = widgets.Dropdown(options=["beatles","RWC"],value = "beatles",description='Dataset:',
                                  layout=widgets.Layout(width='20%'),disabled=False)
dropdown_split = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6, 7],value = 3,description='Split:',
                                  layout=widgets.Layout(width='15%'),disabled=False)
dropdown_id = widgets.Dropdown(description='Track ID:',disabled=False,layout=widgets.Layout(width='20%'))
textbox_track_id = widgets.Text(description='',disabled=True)
button = widgets.Button(description='Load Track')
# widgets for selecting data
button.on_click(process)
selection = widgets.HBox([dropdown_dataset,dropdown_split, dropdown_id,textbox_track_id, button])

# widgets for controlling the plot
range_slider = widgets.FloatRangeSlider(value=[0, 20], min=0, max=10, step=1, description='Time Range:', readout_format='.1f')
beat_alignment = widgets.Checkbox(value=False, description='beat align features', disabled=False)
chroma_type = widgets.RadioButtons(options=['CRP', 'madmom','librosa'], value='CRP', description='Chroma', disabled=False)
fft_length = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6, 7],value = 3,description='Split:',
                                  layout=widgets.Layout(width='15%'),disabled=False)
controls = widgets.HBox([range_slider, beat_alignment, chroma_type])
# Initially hide the controls
controls_container = widgets.VBox([selection])


dropdown_split.observe(update_dropdown_id_options, 'value')
dropdown_id.observe(update_selected_track_id, 'value')

range_slider.observe(on_control_change, 'value')
beat_alignment.observe(on_control_change, 'value')
chroma_type.observe(on_control_change, 'value')
# initialize dropdown menus
update_dropdown_id_options()
update_selected_track_id()
display(controls_container)
display(output)

Output()